#1. 환경구축

##1-1. 텍스트 추출

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver=webdriver.Chrome(options=chrome_options)


In [ ]:
import json
import pandas as pd
import re

## 1-2. 텍스트 전처리

In [ ]:
!pip install pandas
!pip install nltk
!pip install numpy

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import os
import math

In [ ]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
stopwords2=['.',',']

##1-3. vector space model 구축

In [ ]:
!pip install scikit-learn

#2. 텍스트 추출

In [ ]:
url="https://www.gutenberg.org/cache/epub/1342/pg1342.txt"
driver.get(url)
soup=bs(driver.page_source, 'html.parser')

In [ ]:
contents=soup.find('body').string
print(contents[0:50])

In [ ]:
contents_head_cut=contents.split('Chapter I.]')[1].lstrip()
print(contents_head_cut[0:30])

In [ ]:
tail='''                            [Illustration:

                                  THE
                                  END
                                   ]
'''
contents_tail_cut=contents_head_cut.split(tail)[0].rstrip()
print(contents_tail_cut[-30:])

In [ ]:
contents_chapters=contents_tail_cut.split('CHAPTER')

In [ ]:
contents_chapters_cut=[]

contents_chapters_cut.append(contents_chapters[0])

for i in range(1, len(contents_chapters)):
  contents_chapters_cut.append(contents_chapters[i][7:])

In [ ]:
def clean_chapters(chapter):
  new_content=chapter.replace('\n',' ').replace('[Illustration]','')
  pattern1=r'\[Illustration.*?\]'
  #pattern2=r'\[Illustration.*?\]\]'

  new_content=re.sub(pattern1,'',new_content)
  new_content=new_content.replace(' ] ','').replace('  ',' ')

  return new_content

In [ ]:
contents_chapters_cleaned=[]

for chapters in contents_chapters_cut:
  contents_chapters_cleaned.append(clean_chapters(chapters))

In [ ]:
chapters_target=[3,5,7,9,11,13,15,17,19]

target_txt=[]

for target in chapters_target:
  target_txt.append(contents_chapters_cleaned[target-1])

In [ ]:
target_txt[0]

#3. 텍스트 전처리

In [ ]:
def preprocess_with_stemming(text):
  text = re.sub(r'[^\w\s.,?]', '', text.lower())
  tokens = word_tokenize(text)
  stemmer=PorterStemmer()
  tokens_cleaned=[]

  for t in tokens:
    if t not in stopwords:
      if t not in stopwords2:
        tokens_cleaned.append(stemmer.stem(t))


  return tokens_cleaned

In [ ]:
target_with_stemming=[]

for target in target_txt:
  target_with_stemming.append(preprocess_with_stemming(target))

In [ ]:
def preprocess_without_stemming(text):
  text_cleaned = re.sub(r'[^\w\s.,?]', '', text.lower())
  tokens=word_tokenize(text_cleaned)
  tokens_filtered=[]
  for t in tokens:
    if t not in stopwords:
      if t not in stopwords2:
        tokens_filtered.append(t)

  return tokens_filtered

In [ ]:
target_without_stemming=[]

for target in target_txt:
  target_without_stemming.append(preprocess_without_stemming(target))

#4. Vector Space Model 구축

In [ ]:
##테스트
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

preprocessed_documents=[' '.join(doc) for doc in target_without_stemming]
preprocessed_query=' '.join(preprocess_without_stemming("Who was Mr. Bingley's first choice at the party?"))

tfidf_vectorizer=TfidfVectorizer()
tfidf_matrix=tfidf_vectorizer.fit_transform(preprocessed_documents)

query_vector=tfidf_vectorizer.transform([preprocessed_query])

cosine_similarities=cosine_similarity(query_vector, tfidf_matrix)

results=[(chapters_target[i], cosine_similarities[0][i]) for i in range(len(preprocessed_documents))]
results.sort(key=lambda x: x[1], reverse=True)

print(results)

#5. 쿼리 실행